# Estimate Tests

- Create the internal consistency code
- build the diff file code for estimates 

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import copy
import pandas as pd
import pyodbc
import os
import glob
import numpy as np

What internal consistency will do 


Functions: 
1. Function where you pass in the three levels and it checks for equivalence between them and outputs differences, maybe a helpful dataframe 
2. Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 
- Could output a dataframe where all the sums are there then filter for greater than zero, or output the whole dataframe 
3. Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 
4. Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 
5. Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [2]:
def internal_consistency():
    '''
    Along with an individual check for each column sum 

    Total Population for the region, jurisdiction, cpa matches the sum of household population and GQ population 

Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 
    
    '''

# Internal Consistency

In [2]:

# importing the required modules
import glob
import pandas as pd
  
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Raw Data"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

new = {}
for path in files:
    df = pd.read_csv(path)
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis = 1)
        print('Has')
    new[path.split('\\')[1][:-4]] = df

Has
Has
Has


In [3]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [4]:

# importing the required modules
import glob
import pandas as pd
  
# specifying the path to csv files
path = "C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2020_06"
  
# csv files in the path
files = glob.glob(path + "/*.csv")

old = {}
for path in files:
    old[path.split('\\')[1][:-4]] = pd.read_csv(path)

In [5]:
old.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

### Compare Totals 
This is comparing the totals accross CPA, Jur, and Region to see if they are equivalent. 

In [4]:
def compare_totals(cpa_df, jur_df, region_df):
    """
    Aggregates sum of values by year in cpa, jurisdiction, and region level and compares them together to see if they match. CPA is removed as many areas calculated in the other georgraphy 
    """
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df
    
    non_matches_dict = {}
    jur_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(jur_totals[col], cpa_totals[col]).all():
            jur_non_matches.append(col)
                        
    if len(jur_non_matches) == len(cpa_totals.columns):
        non_matches_dict['jurisdiction'] = 'all columns did not match.'
    elif len(jur_non_matches) == 0:
        non_matches_dict['jurisdiction'] = 'all columns matched.'
    else:
        if len(jur_non_matches) > 10:
            non_matches_dict['jurisdiction'] = f"{len(jur_non_matches)} columns did not match out of {len(cpa_totals.columns)} columns."
        else:
            non_matches_dict['jurisdiction'] = ', '.join(jur_non_matches) + ' columns did not match.'
    
    reg_non_matches = []
    for col in cpa_totals.columns:
        if not np.isclose(reg_totals[col], cpa_totals[col]).all():
            reg_non_matches.append(col)
    if len(reg_non_matches) == len(cpa_totals.columns):
        non_matches_dict['region'] = 'all columns did not match.'
    elif len(reg_non_matches) == 0:
        non_matches_dict['region'] = 'all columns matched.'
    else:
        if len(reg_non_matches) > 10:
            non_matches_dict['region'] = f"{len(reg_non_matches)} columns did not match out of {len(cpa_totals.columns)}."
        else:
            non_matches_dict['region'] = ', '.join(reg_non_matches) + ' columns did not match.'
    
    return non_matches_dict

In [5]:
compare_totals(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'jurisdiction': 'all columns matched.', 'region': 'all columns matched.'}

### Ethnicity population check 
Total Population for the region, jurisdiction, cpa matches the sum of all ethnicity group population 

In [6]:
def ethnicity_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Hispanic'] + df['Non-Hispanic, White'] + df['Non-Hispanic, Black'] + df['Non-Hispanic, American Indian or Alaska Native'] + df['Non-Hispanic, Asian'] + df['Non-Hispanic, Hawaiian or Pacific Islander'] + df['Non-Hispanic, Other'] + df['Non-Hispanic, Two or More Races']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results


In [7]:
ethnicity_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Age Group Comparison Check
Total Population for the region, jurisdiction, cpa matches the sum of all age-group population 

In [10]:
def age_group_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Under 5'] + df['5 to 9'] + df['10 to 14'] + df['15 to 17'] + df['18 and 19'] + df['20 to 24'] + df['25 to 29'] + df['30 to 34'] + df['35 to 39'] + df['40 to 44'] + df['45 to 49'] + df['50 to 54'] + df['55 to 59'] + df['60 and 61'] + df['62 to 64'] + df['65 to 69'] + df['70 to 74'] + df['75 to 79'] + df['80 to 84'] + df['85 and Older']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [11]:
age_group_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

### Gender Population test 
Total Population for the region, jurisdiction, cpa matches the sum of male and female population from gender table 

In [12]:
def gender_internal_consistency(cpa_df, jur_df, region_df):
    cpa_totals = cpa_df.groupby('yr_id').sum()
    jur_totals = jur_df.groupby('yr_id').sum()
    reg_totals = region_df


    results = {}

    for geography_data in [('CPA',cpa_totals), ('Jurisdcition',jur_totals), ('Region',reg_totals)]:
        
        geo_level = geography_data[0]
        df = geography_data[1]

        pop_test = df['Female'] + df['Male']

        if sum(~(pop_test == df['Total Population'])) != 0:
            results[geo_level] = 'Ethnicity population does not sum up correctly'
        else:
            results[geo_level] = 'Pass'
    
    return results

In [13]:
gender_internal_consistency(new['2021_01_consolidated_cpa_QA'], new['2021_01_consolidated_jurisdiction_QA'], new['2021_01_consolidated_region_QA'])

{'CPA': 'Pass', 'Jurisdcition': 'Pass', 'Region': 'Pass'}

# Checking for Null Values
Code pulled from Prabina's code with slight modifications.

In [18]:
def spot_nulls(df):
    if (df.isna().sum() != 0).all():
        print('Null values present')
        return df.columns[df.isnull().any()].tolist() # returns the column names that have null vals
    else:
        print('No nulls Present')

In [19]:
spot_nulls(new['2021_01_consolidated_cpa_QA'])

No nulls Present


In [20]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [21]:
spot_nulls(new['2021_01_age_sex_ethnicity_region_QA'])

No nulls Present


# Creating Diff Files

In [29]:
def diff_file(df1, df2, geo_level=None):
    '''
    This function creates a diff file of two dataframes with identical labels and identical shapes and creates a diff file of the two. 
    This does df1 minus df2
    
    '''
    if sum(~(df1.columns == df2.columns)) != 0:
        print('Columns are not identical, cannot preform diff')


    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id']) - df2.set_index(['cpa', 'yr_id'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id']) - df2.set_index(['jurisdiction', 'yr_id'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id']) - df2.set_index(['region', 'yr_id'])).reset_index()

    
    return new_df

In [62]:
# Special for age_sex_ethnicity dataframe
def diff_file(df1, df2, geo_level=None):
    '''
    This function creates a diff file of two dataframes with identical labels and identical shapes and creates a diff file of the two. 
    This does df1 minus df2
    
    '''
    if sum(~(df1.columns == df2.columns)) != 0:
        print('Columns are not identical, cannot preform diff')


    if geo_level == None:
        print('Please input a geography level')
    elif geo_level == 'cpa':
        new_df = (df1.set_index(['cpa', 'yr_id', 'name', 'sex']) - df2.set_index(['cpa', 'yr_id', 'name', 'sex'])).reset_index()    
    elif geo_level == 'jurisdiction':
        new_df = (df1.set_index(['jurisdiction', 'yr_id', 'name', 'sex']) - df2.set_index(['jurisdiction', 'yr_id', 'name', 'sex'])).reset_index()
    elif geo_level == 'region':
        new_df = (df1.set_index(['region', 'yr_id', 'name', 'sex']) - df2.set_index(['region', 'yr_id', 'name', 'sex'])).reset_index()

    
    return new_df

In [7]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [8]:
old.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

In [182]:
table = diff_file(new['2021_01_age_sex_ethnicity_region_QA'], old['2020_06_age_sex_ethnicity_region_QA'], geo_level='region')
table

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, White","Non-Hispanic, Asian","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Other","Non-Hispanic, Two or More Races","Non-Hispanic, Black"
0,San Diego,2010,10 to 14,Female,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,San Diego,2010,10 to 14,Male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,San Diego,2010,15 to 17,Female,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,San Diego,2010,15 to 17,Male,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,San Diego,2010,18 and 19,Female,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
475,San Diego,2021,80 to 84,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
476,San Diego,2021,85 and Older,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
477,San Diego,2021,85 and Older,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
478,San Diego,2021,Under 5,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
with pd.ExcelWriter("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Diff Files/age_sex_ethniicty_diff_region_QA.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    new['2021_01_age_sex_ethnicity_region_QA'].to_excel(writer, sheet_name="2021_01", index=False)
    old['2020_06_age_sex_ethnicity_region_QA'].to_excel(writer, sheet_name="2020_06", index=False)
    table.to_excel(writer, sheet_name="Diff - 2021_01 minus 2020_06", index=False)

# Year on Year output file
- Is this meant to be for every file that we have? Or just the consolidated ones

In [37]:
def yoy_pct_diff(df, geo_level=None):
    '''
    This finds the percent change by year
    df must be already indexed
    
    '''
    if geo_level == None:
        print('Please input a valid geography level')
    elif geo_level == 'region':
        final = df.pct_change()*100
    elif geo_level == 'cpa':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    elif geo_level == 'jurisdiction':
        final = pd.DataFrame()
        for geography in list(set(df.index.get_level_values(0))):
            final = final.append(df.loc[[geography]].pct_change()*100)
    return final

In [38]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [39]:
old.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

In [49]:
yoy_pct_diff(old['2020_06_consolidated_region_QA'].set_index(['region', 'yr_id']), geo_level='region').to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2020_06/yoy_pct_change/consolidated_region_change.csv')

# Proportion Check 
- Compare the proportion of groups (age/gender/ethnicity/ethnicity by gender by age in total population between DOF and Estimates at Region Level  

- To be performed on Age, Ethnicity, Ethnicity by age and gender, household income 

- Gender still needs to be outputted 


- What is category?
- Am I doing all of the proportion checks?



1. household income

2. age

3. gender (later)

4. ethnicity

5. ethnicity by age

6. ethnicity by age by gender



- Ask bryce which DOF table the current estimates use?
- Where is the dimension table for race_code, same as in demographic_warehouse?

In chat: 
- Start chat with whoever is in that group 


In [83]:
data.keys()

dict_keys(['2020_06_age_cpa_QA', '2020_06_age_ethnicity_cpa_QA', '2020_06_age_ethnicity_jurisdiction_QA', '2020_06_age_ethnicity_region_QA', '2020_06_age_jurisdiction_QA', '2020_06_age_region_QA', '2020_06_age_sex_ethnicity_cpa_QA', '2020_06_age_sex_ethnicity_jurisdiction_QA', '2020_06_age_sex_ethnicity_region_QA', '2020_06_consolidated_cpa_QA', '2020_06_consolidated_jurisdiction_QA', '2020_06_consolidated_region_QA', '2020_06_ethnicity_cpa_QA', '2020_06_ethnicity_jurisdiction_QA', '2020_06_ethnicity_region_QA', '2020_06_household_income_cpa_QA', '2020_06_household_income_jurisdiction_QA', '2020_06_household_income_region_QA'])

In [86]:
data['2020_06_age_ethnicity_region_QA'].set_index(['region', 'yr_id', 'name'])


'''
1st way: 
90k / sum(10-14 age group for all ethnicity)

2nd way
90k/ total hispanic population

'''

Hispanic  Non-Hispanic, White  \
region    yr_id name                                          
San Diego 2010  10 to 14         90039                68297   
                15 to 17         57838                44692   
                18 and 19        39688                37261   
                20 to 24         93539               119871   
                25 to 29         84428               114319   
...                                ...                  ...   
          2020  70 to 74         22879                88779   
                75 to 79         15173                59109   
                80 to 84         10510                39244   
                85 and Older     10061                44814   
                Under 5          86870                80213   

                              Non-Hispanic, Asian  \
region    yr_id name                                
San Diego 2010  10 to 14                    17950   
                15 to 17                    11065   
                18 and 19                    9922   
                20 to 24                    29194   
                25 to 29                    27797   
...                                           ...   
          2020  70 to 74                    15255   
                75 to 79                    10214   
                80 to 84                     7518   
                85 and Older                 7162   
                Under 5                     18202   

                              Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name                                                       
San Diego 2010  10 to 14                                             819   
                15 to 17                                             579   
                18 and 19                                            472   
                20 to 24                                            1439   
                25 to 29                                            1235   
...                                                                  ...   
          2020  70 to 74                                             506   
                75 to 79                                             257   
                80 to 84                                             280   
                85 and Older                                         230   
                Under 5                                              729   

                              Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name                                                           
San Diego 2010  10 to 14                                                 871   
                15 to 17                                                 605   
                18 and 19                                                501   
                20 to 24                                                1496   
                25 to 29                                                1203   
...                                                                      ...   
          2020  70 to 74                                                 601   
                75 to 79                                                 340   
                80 to 84                                                 236   
                85 and Older                                             216   
                Under 5                                                  939   

                              Non-Hispanic, Other  \
region    yr_id name                                
San Diego 2010  10 to 14                      524   
                15 to 17                      344   
                18 and 19                     223   
                20 to 24                      615   
                25 to 29                      726   
...                                           ...   
          2020  70 to 74                      161   
             

In [84]:
data['2020_06_household_income_region_QA']
# formula value / sum(values)

,region,yr_id,"Less than $15,000","$15,000 to $29,999","$30,000 to $44,999","$45,000 to $59,999","$60,000 to $74,999","$75,000 to $99,999","$100,000 to $124,999","$125,000 to $149,999","$150,000 to $199,999","$200,000 or more"
0,San Diego,2010,117399,149059,153011,123744,111343,142638,100598,63023,64067,61984
1,San Diego,2011,117657,149510,153502,124118,111639,142889,100769,63130,64157,62050
2,San Diego,2012,134432,155319,146936,127768,109248,138737,97464,59502,63620,60916
3,San Diego,2013,131390,160423,151880,128231,106684,138648,94698,60000,68240,63338
4,San Diego,2014,128661,159289,152979,130555,105186,136495,99016,58772,66468,72729
5,San Diego,2015,124967,150445,148877,126271,105363,145985,104614,60883,69431,80806
6,San Diego,2016,115619,147924,144916,133206,110768,151444,107571,63025,69472,81639
7,San Diego,2017,105544,146182,145843,128844,112160,148686,111103,66105,78477,91317
8,San Diego,2018,100975,141923,136894,128215,111662,153338,114798,68592,83483,103620
9,San Diego,2019,105792,136944,138600,128470,108525,155468,115643,71304,88240,104390


# Proportion Checks 

In [67]:
new.keys()

dict_keys(['2021_01_age_cpa_QA', '2021_01_age_ethnicity_cpa_QA', '2021_01_age_ethnicity_jurisdiction_QA', '2021_01_age_ethnicity_region_QA', '2021_01_age_jurisdiction_QA', '2021_01_age_region_QA', '2021_01_age_sex_ethnicity_cpa_QA', '2021_01_age_sex_ethnicity_jurisdiction_QA', '2021_01_age_sex_ethnicity_region_QA', '2021_01_consolidated_cpa_QA', '2021_01_consolidated_jurisdiction_QA', '2021_01_consolidated_region_QA', '2021_01_ethnicity_cpa_QA', '2021_01_ethnicity_jurisdiction_QA', '2021_01_ethnicity_region_QA', '2021_01_households_cpa_QA', '2021_01_households_jurisdiction_QA', '2021_01_households_region_QA', '2021_01_household_income_cpa_QA', '2021_01_household_income_jurisdiction_QA', '2021_01_household_income_region_QA'])

In [170]:
new['2021_01_age_sex_ethnicity_region_QA']

,region,yr_id,name,sex,Hispanic,"Non-Hispanic, White","Non-Hispanic, Asian","Non-Hispanic, Hawaiian or Pacific Islander","Non-Hispanic, American Indian or Alaska Native","Non-Hispanic, Other","Non-Hispanic, Two or More Races","Non-Hispanic, Black"
0,San Diego,2010,10 to 14,Female,44161,33071,8634,385,398,254,5133,4777
1,San Diego,2010,10 to 14,Male,45878,35226,9316,434,473,270,5233,5073
2,San Diego,2010,15 to 17,Female,27852,21611,5315,270,291,156,2903,3260
3,San Diego,2010,15 to 17,Male,29986,23081,5750,309,314,188,3008,3706
4,San Diego,2010,18 and 19,Female,18981,16601,4888,204,216,103,1846,2290
...,...,...,...,...,...,...,...,...,...,...,...,...
475,San Diego,2021,80 to 84,Male,4092,17807,3038,114,118,32,303,936
476,San Diego,2021,85 and Older,Female,6828,28126,4905,140,134,33,436,1115
477,San Diego,2021,85 and Older,Male,3633,17308,2481,107,94,18,260,735
478,San Diego,2021,Under 5,Female,39979,37907,8645,334,425,230,4774,4657


In [171]:
df = new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum()
sum_vals = list(df.sum(axis=1))
row_sum_percentage = df.div(sum_vals, axis=0)*100
row_sum_percentage

Hispanic  Non-Hispanic, White  \
region    yr_id name         sex                                      
San Diego 2010  10 to 14     Female  45.614742            34.159669   
                             Male    45.021246            34.568168   
                15 to 17     Female  45.171754            35.049791   
                             Male    45.199120            34.790932   
                18 and 19    Female  42.059430            36.785659   
...                                        ...                  ...   
          2021  80 to 84     Male    15.476551            67.348714   
                85 and Older Female  16.367428            67.420955   
                             Male    14.746712            70.254912   
                Under 5      Female  41.236295            39.099133   
                             Male    40.848106            39.466706   

                                     Non-Hispanic, Asian  \
region    yr_id name         sex                           
San Diego 2010  10 to 14     Female             8.918224   
                             Male               9.142027   
                15 to 17     Female             8.620130   
                             Male               8.667209   
                18 and 19    Female            10.831173   
...                                                  ...   
          2021  80 to 84     Male              11.490166   
                85 and Older Female            11.757797   
                             Male              10.070628   
                Under 5      Female             8.916876   
                             Male               8.775014   

                                     Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name         sex                                                  
San Diego 2010  10 to 14     Female                                    0.397674   
                             Male                                      0.425895   
                15 to 17     Female                                    0.437899   
                             Male                                      0.465768   
                18 and 19    Female                                    0.452037   
...                                                                         ...   
          2021  80 to 84     Male                                      0.431165   
                85 and Older Female                                    0.335595   
                             Male                                      0.434324   
                Under 5      Female                                    0.344504   
                             Male                                      0.355529   

                                     Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name         sex                                                      
San Diego 2010  10 to 14     Female                                        0.411102   
                             Male                                          0.464167   
                15 to 17     Female                                        0.471958   
                             Male                                          0.473305   
                18 and 19    Female                                        0.478628   
...                                                                             ...   
          2021  80 to 84     Male                                          0.446293   
                85 and Older Female                                        0.321212   
                             Male                                          0.381555   
                Under 5      Female                                        0.438366   
                             Male                                          0.514425   

                                     Non-Hispanic, Other  \
region    yr_id name         sex                           
S

In [ ]:
df = new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum()
sum_vals = list(df.sum(axis=1))
row_sum_percentage = df.div(sum_vals, axis=0)*100
row_sum_percentage

In [172]:
row_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_row_wise_sums.csv')

In [129]:
df = new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum()
sum_vals = list(df.sum(axis=0))
col_sum_percentage = df.div(sum_vals, axis=1)*100
col_sum_percentage

Hispanic  Non-Hispanic, White  \
region    yr_id name                                          
San Diego 2010  10 to 14      0.693788             0.372994   
                15 to 17      0.445666             0.244079   
                18 and 19     0.305812             0.203495   
                20 to 24      0.720757             0.654657   
                25 to 29      0.650553             0.624336   
...                                ...                  ...   
          2021  70 to 74      0.185246             0.499337   
                75 to 79      0.121653             0.336403   
                80 to 84      0.082648             0.219475   
                85 and Older  0.080606             0.248131   
                Under 5       0.624994             0.424063   

                              Non-Hispanic, Asian  \
region    yr_id name                                
San Diego 2010  10 to 14                 0.433983   
                15 to 17                 0.267522   
                18 and 19                0.239887   
                20 to 24                 0.705832   
                25 to 29                 0.672056   
...                                           ...   
          2021  70 to 74                 0.385580   
                75 to 79                 0.253040   
                80 to 84                 0.187060   
                85 and Older             0.178574   
                Under 5                  0.422643   

                              Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name                                                       
San Diego 2010  10 to 14                                        0.480541   
                15 to 17                                        0.339723   
                18 and 19                                       0.276942   
                20 to 24                                        0.844320   
                25 to 29                                        0.724625   
...                                                                  ...   
          2021  70 to 74                                        0.318014   
                75 to 79                                        0.160180   
                80 to 84                                        0.156660   
                85 and Older                                    0.144925   
                Under 5                                         0.406025   

                              Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name                                                           
San Diego 2010  10 to 14                                            0.492421   
                15 to 17                                            0.342038   
                18 and 19                                           0.283241   
                20 to 24                                            0.845766   
                25 to 29                                            0.680118   
...                                                                      ...   
          2021  70 to 74                                            0.357868   
                75 to 79                                            0.196742   
                80 to 84                                            0.138511   
                85 and Older                                        0.128900   
                Under 5                                             0.533127   

                              Non-Hispanic, Other  \
region    yr_id name                                
San Diego 2010  10 to 14                 0.607473   
                15 to 17                 0.398799   
                18 and 19                0.258524   
                20 to 24                 0.712969   
                25 to 29                 0.841651   
...                                           ...   
          2021  70 to 74                 0.173895   
             

In [ ]:
# # Ethnicity by age code

# final = pd.DataFrame()


# for year in list(set(new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum().index.get_level_values(1))):

#     df = new['2021_01_age_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name']).sum().loc[("San Diego", year)]


#     sum_vals = list(df.sum(axis=0))
#     col_sum_percentage = df.div(sum_vals, axis=1)*100

#     df = col_sum_percentage


#     df['region'] = 'San Diego'
#     df['yr_id'] = year


#     df = df.reset_index()
#     final = final.append(df)


# # final.groupby(['region', 'yr_id', 'name']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/ethnicity_by_age_col_wise_sums.csv')

In [173]:
new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum()

Hispanic  Non-Hispanic, White  \
region    yr_id name         sex                                     
San Diego 2010  10 to 14     Female     44161                33071   
                             Male       45878                35226   
                15 to 17     Female     27852                21611   
                             Male       29986                23081   
                18 and 19    Female     18981                16601   
...                                       ...                  ...   
          2021  80 to 84     Male        4092                17807   
                85 and Older Female      6828                28126   
                             Male        3633                17308   
                Under 5      Female     39979                37907   
                             Male       41132                39741   

                                     Non-Hispanic, Asian  \
region    yr_id name         sex                           
San Diego 2010  10 to 14     Female                 8634   
                             Male                   9316   
                15 to 17     Female                 5315   
                             Male                   5750   
                18 and 19    Female                 4888   
...                                                  ...   
          2021  80 to 84     Male                   3038   
                85 and Older Female                 4905   
                             Male                   2481   
                Under 5      Female                 8645   
                             Male                   8836   

                                     Non-Hispanic, Hawaiian or Pacific Islander  \
region    yr_id name         sex                                                  
San Diego 2010  10 to 14     Female                                         385   
                             Male                                           434   
                15 to 17     Female                                         270   
                             Male                                           309   
                18 and 19    Female                                         204   
...                                                                         ...   
          2021  80 to 84     Male                                           114   
                85 and Older Female                                         140   
                             Male                                           107   
                Under 5      Female                                         334   
                             Male                                           358   

                                     Non-Hispanic, American Indian or Alaska Native  \
region    yr_id name         sex                                                      
San Diego 2010  10 to 14     Female                                             398   
                             Male                                               473   
                15 to 17     Female                                             291   
                             Male                                               314   
                18 and 19    Female                                             216   
...                                                                             ...   
          2021  80 to 84     Male                                               118   
                85 and Older Female                                             134   
                             Male                                                94   
                Under 5      Female                                             425   
                             Male                                               518   

                                     Non-Hispanic, Other  \
region    yr_id name         sex                           
San Diego 201

In [174]:
new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().loc[("San Diego", year)]

Hispanic  Non-Hispanic, White  Non-Hispanic, Asian  \
name         sex                                                          
10 to 14     Female     44593                33337                 9185   
             Male       46629                35573                 9748   
15 to 17     Female     27717                22200                 5860   
             Male       30147                24607                 6424   
18 and 19    Female     20168                17700                 4817   
             Male       17331                25105                 4489   
20 to 24     Female     49977                42126                11369   
             Male       60881                56513                13408   
25 to 29     Female     38642                44723                12144   
             Male       41283                50485                12580   
30 to 34     Female     40324                51636                13877   
             Male       41464                57383                13200   
35 to 39     Female     37611                44525                14067   
             Male       36604                50299                12456   
40 to 44     Female     37897                43142                15104   
             Male       35021                49120                12554   
45 to 49     Female     34911                46986                13753   
             Male       32894                50933                11502   
5 to 9       Female     48667                36933                 9372   
             Male       50219                38836                 9823   
50 to 54     Female     30858                56253                12911   
             Male       28564                59718                10890   
55 to 59     Female     25214                59207                12411   
             Male       22148                60804                 9463   
60 and 61    Female      8213                23061                 4687   
             Male        6970                22531                 3662   
62 to 64     Female     10239                31941                 6477   
             Male        8635                31204                 4873   
65 to 69     Female     13568                48423                 9382   
             Male       10884                46416                 6487   
70 to 74     Female      9890                33805                 6313   
             Male        6832                31368                 4548   
75 to 79     Female      7544                25494                 5263   
             Male        4946                21574                 3551   
80 to 84     Female      5483                20497                 3890   
             Male        3348                16042                 2173   
85 and Older Female      5133                27904                 3285   
             Male        2887                16245                 1842   
Under 5      Female     49733                39338                 9238   
             Male       51666                41611                 9875   

                     Non-Hispanic, Hawaiian or Pacific Islander  \
name         sex                                                  
10 to 14     Female                                         369   
             Male                                           399   
15 to 17     Female                                         253   
             Male                                           341   
18 and 19    Female                                         198   
             Male                                           299   
20 to 24     Female                                         496   
             Male                                           688   
25 to 29     Female                                         535   
             Male                                           613   
30 to 34     Female                                         587   

In [176]:
# # Age Sex Ethnicity Code 

# final = pd.DataFrame()


# for year in list(set(new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().index.get_level_values(1))):

#     df = new['2021_01_age_sex_ethnicity_region_QA'].groupby(['region', 'yr_id', 'name', 'sex']).sum().loc[("San Diego", year)]


#     sum_vals = list(df.sum(axis=0))
#     col_sum_percentage = df.div(sum_vals, axis=1)*100

#     df = col_sum_percentage


#     df['region'] = 'San Diego'
#     df['yr_id'] = year


#     df = df.reset_index()
#     final = final.append(df)


# final.groupby(['region', 'yr_id', 'name', 'sex']).sum()


# final.groupby(['region', 'yr_id', 'name', 'sex']).sum().to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/age_sex_ethnicity_col_wise_sums.csv')

In [130]:
col_sum_percentage.to_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/2021_01/Proportion Checks/ethnicity_by_age_col_wise_sums.csv')

In [106]:
df

Less than $15,000  $15,000 to $29,999  $30,000 to $44,999  \
region    yr_id                                                              
San Diego 2010              117399              149059              153011   
          2011              117655              149507              153501   
          2012              134337              155257              146891   
          2013              131072              160093              151607   
          2014              128320              158893              152621   
          2015              124626              150080              148524   
          2016              115110              147323              144357   
          2017              105079              145561              145220   
          2018              100413              141203              136210   
          2019              105174              136187              137874   
          2020              102652              125813              132953   
          2021              104751              128294              135378   

                 $45,000 to $59,999  $60,000 to $74,999  $75,000 to $99,999  \
region    yr_id                                                               
San Diego 2010               123744              111343              142638   
          2011               124115              111644              142889   
          2012               127719              109220              138667   
          2013               127953              106486              138387   
          2014               130246              104963              136209   
          2015               125962              105111              145658   
          2016               132673              110314              150925   
          2017               128290              111683              148071   
          2018               127594              111101              152554   
          2019               127806              107921              154683   
          2020               124422              111506              160243   
          2021               126076              112867              161609   

                 $100,000 to $124,999  $125,000 to $149,999  \
region    yr_id                                               
San Diego 2010                 100598                 63023   
          2011                 100770                 63130   
          2012                  97433                 59492   
          2013                  94533                 59892   
          2014                  98783                 58638   
          2015                 104344                 60748   
          2016                 107191                 62794   
          2017                 110661                 65851   
          2018                 114247                 68256   
          2019                 115043                 70938   
          2020                 122344                 68015   
          2021                 122956                 66184   

                 $150,000 to $199,999  $200,000 or more  
region    yr_id                                          
San Diego 2010                  64066             61985  
          2011                  64158             62052  
          2012                  63596             60900  
          2013                  68119             63211  
          2014                  66331             72559  
          2015                  69260             80597  
          2016                  69222             81334  
          2017                  78144             90905  
          2018                  83083            103130  
          2019                  87781            103869  
          2020                  88224            117809  
          2021                  87013            115644

In [109]:
sum_vals

[1386588,
 1747270,
 1738147,
 1526600,
 1314159,
 1772533,
 1288903,
 766961,
 888997,
 1013995]

In [95]:
row_sum_percentage = df.div(sum_vals, axis=0)

In [80]:
new['2021_01_age_cpa_QA'].div(list(new['2021_01_age_cpa_QA'].groupby(['cpa', 'yr_id']).sum().sum(axis=1)), axis=0)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=socioec_data;'
                    'Trusted_Connection=yes;')

In [3]:
age_query = '''SELECT * 
FROM [ddamwsql16].[socioec_data].[ca_dof].[population_proj_2021_07_14]'''

age_df =  pd.read_sql_query(age_query, conn) #example df

In [82]:
age_df

,county_fips_code,fiscal_yr,race_code,sex,age,population
0,6001,2010,1,F,0,2217
1,6001,2010,1,F,1,2038
2,6001,2010,1,F,2,2008
3,6001,2010,1,F,3,2129
4,6001,2010,1,F,4,2013
...,...,...,...,...,...,...
4596727,6115,2060,7,M,106,0
4596728,6115,2060,7,M,107,0
4596729,6115,2060,7,M,108,0
4596730,6115,2060,7,M,109,0


### Age Proportion Check

In [81]:
data['2020_06_consolidated_region_QA']

,region,yr_id,Under 5,5 to 9,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,...,Mobile Home,Single-family Detached,Single-family Attached,Total Population,Household Population,Group Quarters - Military,Group Quarters - College,Group Quarters - Other,Female,Male
0,San Diego,2010,203423,194029,198716,128000,97095,270750,250737,220185,...,39846,0,0,3095314,2993348,43597,16055,42314,1541635,1553679
1,San Diego,2011,209818,197773,198866,128833,95984,269851,246589,224827,...,39956,0,0,3125655,3023755,42309,16968,42623,1555437,1570218
2,San Diego,2012,215677,203754,198403,131596,97211,267545,240615,230265,...,40058,0,0,3161809,3058317,42539,17540,43413,1572513,1589296
3,San Diego,2013,219753,210910,199295,133282,98736,265080,234627,235654,...,40119,0,0,3199901,3098470,40622,18316,42493,1590776,1609125
4,San Diego,2014,222064,216798,200875,132669,100101,263770,228231,240016,...,40168,0,0,3232763,3127530,42313,19046,43874,1606047,1626716
5,San Diego,2015,223824,221252,202795,131361,101520,263884,222382,240792,...,40169,0,0,3265039,3156961,43732,19357,44989,1620812,1644227
6,San Diego,2016,223355,224985,205244,131064,100003,263975,218370,236207,...,40222,0,0,3285151,3177681,44193,19160,44117,1630016,1655135
7,San Diego,2017,221684,228181,210062,131314,98316,264924,215777,230157,...,40081,0,0,3306890,3200591,43280,18989,44030,1640232,1666658
8,San Diego,2018,219100,229534,215718,131683,98570,264797,212733,223717,...,40020,0,0,3326318,3215545,43995,20178,46600,1649257,1677061
9,San Diego,2019,213921,230600,220580,132502,99494,263897,210274,216586,...,40116,0,0,3340312,3230523,44257,20208,45324,1655636,1684676


In [ ]:
'''SELECT dof.fiscal_yr AS 'Year', dof.sex, dim.ethnicity_id
FROM [ddamwsql16].[socioec_data].[ca_dof].[population_proj_2021_07_14] AS dof
LEFT JOIN
	(SELECT [ethnicity_id], [short_name]
	FROM [estimates].[est_2020_06].[dw_age]) AS dim
	ON (dof.[race_code] = dim.[ethnicity_id])'''